In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from time import sleep

In [3]:
spark = SparkSession.builder.appName('Basic Stream').getOrCreate()

In [4]:
fileSchema = spark.read.json('D:/Dataset/activity-data/')
fileSchema.printSchema()

root
 |-- Arrival_Time: long (nullable = true)
 |-- Creation_Time: long (nullable = true)
 |-- Device: string (nullable = true)
 |-- Index: long (nullable = true)
 |-- Model: string (nullable = true)
 |-- User: string (nullable = true)
 |-- gt: string (nullable = true)
 |-- x: double (nullable = true)
 |-- y: double (nullable = true)
 |-- z: double (nullable = true)



In [5]:
type(fileSchema)

pyspark.sql.dataframe.DataFrame

In [6]:
dataSchema = fileSchema.schema
act_df = spark.readStream.schema(dataSchema).option('maxFilesPerTrigger', 1).json('D:/Dataset/activity-data/')

In [ ]:
spark.conf.set('spark.sql.shuffle.partitions', 1)
act_count = act_df.groupBy('gt').count()
act_querry = act_count.writeStream.queryName('ActivityQuery2') \
    .format('memory') \
    .outputMode('complete') \
    .start()

for x in range (6):
    spark.sql('SELECT * FROM ActivityQuery2').show()
    sleep(5)
    
act_querry.awaitTermination()


+---+-----+
| gt|count|
+---+-----+
+---+-----+

+----------+-----+
|        gt|count|
+----------+-----+
|       sit|12309|
|      walk|13256|
|  stairsup|10452|
|      bike|10796|
|      null|10449|
|     stand|11384|
|stairsdown| 9365|
+----------+-----+

+----------+-----+
|        gt|count|
+----------+-----+
|  stairsup|31357|
|     stand|34154|
|       sit|36929|
|      walk|39768|
|      bike|32390|
|      null|31343|
|stairsdown|28094|
+----------+-----+

+----------+-----+
|        gt|count|
+----------+-----+
|  stairsup|52260|
|     stand|56924|
|       sit|61547|
|      walk|66280|
|      bike|53984|
|      null|52239|
|stairsdown|46825|
+----------+-----+

+----------+-----+
|        gt|count|
+----------+-----+
|  stairsup|62710|
|     stand|68309|
|       sit|73855|
|      walk|79536|
|      bike|64781|
|      null|62688|
|stairsdown|56192|
+----------+-----+

+----------+------+
|        gt| count|
+----------+------+
|  stairsup| 83614|
|     stand| 91079|
|       sit